In [1]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first




def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def primesToN(n):  # lists primes p w. 2 <= p <= n
    answer=[]
    for k in [1..n-1]:
        nk=next_prime(k)
        if (nk in answer)==False:
            if nk<n+1:
                answer=answer+[next_prime(k)]
    return answer

In [15]:
print primesToN(12)

[2, 3, 5, 7, 11]


In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [79]:
abs(-1)

1

In [80]:
# Clause 2 (a) (i)
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if abs(lp-pp)>1:data=data+[index]
print data

[]


In [82]:
#One direction of clause 2 (a) (ii)
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if lp<pp:
      
        data=data+[index]
print data
# data = odd prime numbers


[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]


In [90]:
#Other direction of clause 2 (a) (ii)
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if is_prime(index) & (index > 2):
        data=data+[sign(lp-pp)]
print data

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [98]:
# clause 2(a) (iii)
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
tf=True
for k in [0..len(s)-1]:
    index=s[k][0]
    
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if index ==-1:print (-1,lp,pp)
    if index ==0:print (0,lp,pp)
    if index > 0:
        if lp==pp:
            test=factorial(index)/((index*(index+1))/2)
            tf = tf & (test in ZZ)
print tf

(-1, 0, 0)
(0, 1, 0)
True


In [96]:
# clause 2 (a) (iv)
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    ppi=prime_pi(index)
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= sort(primeFactorsInList(dens))
    lnth=len(pf)
    for j in [0..lnth-2]:
        p1=pf[j]
        nxt=next_prime(p1)
        p2=pf[j+1]
        if nxt != p2:data=data+[(index,p1,p2,nxt)]
print data 
# data empty says: no missing primes within $\delta_n$ (cn1 clause 2)
        

[]


In [43]:
# clause 2 (a) (v) (A) & (B)
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

for k in [0..len(s)-1]:
    index=s[k][0]
    if index==-1:
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= primeFactorsInList(dens)
        
        print (index,pf)
    if index==0:
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= primeFactorsInList(dens)
      
        print (index,pf)


(-1, [])
(0, [2])


In [6]:
# clause 2 (a) (vi)
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
    if max(pf)==index:data=data+[index]
print data

[]


In [4]:
# clause 2 (a) (v) (C)
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    if index>0:
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= primeFactorsInList(dens)
        pp=prime_pi(index+1)
        mpf=max(pf)
        if mpf>index:data=data+[is_prime(index+1)]
print data
signdata=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    if index>0:
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= primeFactorsInList(dens)
        pp=prime_pi(index+1)
        mpf=max(pf)
        if is_prime(index+1):signdata=signdata+[sign(mpf-index)]
print
print signdata

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [26]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
diffs=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if lp<pp:
        diffs=diffs+[pp-lp]
        data=data+[index]
print data
# data = odd prime numbers
print diffs

[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [27]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if lp==pp:
        data=data+[index]
print data # uninformative, but if I segregate the indices 
# by parity I see something (below cells)


[-1, 8, 9, 14, 15, 20, 21, 24, 25, 26, 27, 32, 33, 34, 35, 38, 39, 44, 45, 48, 49, 50, 51, 54, 55, 56, 57, 62, 63, 64, 65, 68, 69, 74, 75, 76, 77, 80, 81, 84, 85, 86, 87, 90, 91, 92, 93, 94, 95, 98, 99, 104, 105, 110, 111, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 128, 129, 132, 133, 134, 135, 140, 141, 142, 143, 144, 145, 146, 147, 152, 153, 154, 155, 158, 159, 160, 161, 164, 165, 168, 169, 170, 171, 174, 175, 176, 177, 182, 183, 184, 185, 186, 187, 188, 189, 194, 195]


In [20]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data0=[]

data1=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if lp==pp:
        if mod(index,2)==0:
            data0=data0+[index]
            
        if mod(index,2)==1:
            data1=data1+[index]
            
print data0;print;print data1
# data0: A309355
# Even numbers k with the property that k! is divisible 
# by k*(k+1)/2.
# & even numbers that are not a prime - 1.
# data1 beginning with 9: odd composite numbers. A071904

[8, 14, 20, 24, 26, 32, 34, 38, 44, 48, 50, 54, 56, 62, 64, 68, 74, 76, 80, 84, 86, 90, 92, 94, 98, 104, 110, 114, 116, 118, 120, 122, 124, 128, 132, 134, 140, 142, 144, 146, 152, 154, 158, 160, 164, 168, 170, 174, 176, 182, 184, 186, 188, 194]

[-1, 9, 15, 21, 25, 27, 33, 35, 39, 45, 49, 51, 55, 57, 63, 65, 69, 75, 77, 81, 85, 87, 91, 93, 95, 99, 105, 111, 115, 117, 119, 121, 123, 125, 129, 133, 135, 141, 143, 145, 147, 153, 155, 159, 161, 165, 169, 171, 175, 177, 183, 185, 187, 189, 195]


In [35]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
tf=True
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if lp==pp:
        if index>-1: #no division by zero message just means lp!=pp at index = 0
            test=factorial(index)/((index*(index+1))/2)
            tf = tf & (test in ZZ)
print tf


True


In [38]:
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
    lp=len(pf)
    pp=prime_pi(index)
    if index>0:
        test=factorial(index)/((index*(index+1))/2)
        if test in ZZ:
            if lp!=pp:no=no+[index]
print no  # = 1 & odd primes

[1, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]


In [61]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
diffs=[]
for k in [2..5]:
    print "==========================================="
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
    lp=len(pf)
    pp=prime_pi(index)
    if lp > pp:
        print (index,lp,pp,prime_n(index))
        print poly
        print pf

(1, 1, 0, 2)
69/1024*x^4 - 1/128*x^2 - 3/64
[2]
(2, 2, 1, 3)
1/128*x^6 - 29/864*x^4 + 1/216*x^2 + 1/54
[2, 3]
(4, 3, 2, 7)
3/65536*x^10 - 16367/18432000*x^8 + 7961/1382400*x^6 - 21809/1728000*x^4 + 373/172800*x^2 + 373/72000
[2, 3, 5]


In [78]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def testf(n):
    if n==0:return 1
    if n>0: return prime_n(n)
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
data2=[]
data3=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
   
    lp=len(pf)
    pp=prime_pi(index)
    if lp>pp&index>-1:
        data=data+[index]
        data2=data2 +[1+index]
        data3=data3+[testf(index)]
print data;print;print data2;print;print data3

[0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 58, 60, 61, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 168, 169, 170, 171, 172, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 188, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199]

[1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 59, 61, 62, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 

In [2]:
max([2,3,6])

6

In [38]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= primeFactorsInList(dens)
    pp=prime_pi(index)
    pn=primes_first_n(pp)
    lp=len(pf)
    if lp<pp:
        data=data+[index]
        lastdenominatorprime=max(pf)
        lastprimeleqindex=max(pn)
        print (index,pp-lp,lastdenominatorprime,lastprimeleqindex)
print data # data = odd primes      



(3, 1, 2, 3)
(5, 1, 3, 5)
(7, 1, 5, 7)
(11, 1, 7, 11)
(13, 1, 11, 13)
(17, 1, 13, 17)
(19, 1, 17, 19)
(23, 1, 19, 23)
(29, 1, 23, 29)
(31, 1, 29, 31)
(37, 1, 31, 37)
(41, 1, 37, 41)
(43, 1, 41, 43)
(47, 1, 43, 47)
(53, 1, 47, 53)
(59, 1, 53, 59)
(61, 1, 59, 61)
(67, 1, 61, 67)
(71, 1, 67, 71)
(73, 1, 71, 73)
(79, 1, 73, 79)
(83, 1, 79, 83)
(89, 1, 83, 89)
(97, 1, 89, 97)
(101, 1, 97, 101)
(103, 1, 101, 103)
(107, 1, 103, 107)
(109, 1, 107, 109)
(113, 1, 109, 113)
(127, 1, 113, 127)
(131, 1, 127, 131)
(137, 1, 131, 137)
(139, 1, 137, 139)
(149, 1, 139, 149)
(151, 1, 149, 151)
(157, 1, 151, 157)
(163, 1, 157, 163)
(167, 1, 163, 167)
(173, 1, 167, 173)
(179, 1, 173, 179)
(181, 1, 179, 181)
(191, 1, 181, 191)
(193, 1, 191, 193)
(197, 1, 193, 197)
(199, 1, 197, 199)
[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]


In [14]:
#listdens(lst)
#f.coefficients() # gives
#primeFactorsIn(lst)
def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data
    
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
data2=[]
data3=[]
data4=[]
for k in [1..len(s)-1]:
    index=s[k][0]
    ppi=prime_pi(index)
    poly=s[k][1]
    cffs=poly.list()
    dz=dropzeros(cffs)
    dens=listdens(dz)
    pf= sort(primeFactorsInList(dens))
    lnth=len(pf)
    if lnth<ppi:data2=data2+[index]
    if lnth==ppi:data3=data3+[index]
    if lnth>ppi:data4=data4+[index+1]
    for j in [0..lnth-2]:
        p1=pf[j]
        nxt=next_prime(p1)
        p2=pf[j+1]
        if nxt != p2:data=data+[(index,p1,p2,nxt)]
print data 
print
print data2 # odd primes A000040
print
print data3
print
print data4 # 1 & primes
# data empty says: no missing primes within $\delta_n$ (cn1 clause 2)
        

[]

[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]

[8, 9, 14, 15, 20, 21, 24, 25, 26, 27, 32, 33, 34, 35, 38, 39, 44, 45, 48, 49, 50, 51, 54, 55, 56, 57, 62, 63, 64, 65, 68, 69, 74, 75, 76, 77, 80, 81, 84, 85, 86, 87, 90, 91, 92, 93, 94, 95, 98, 99, 104, 105, 110, 111, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 128, 129, 132, 133, 134, 135, 140, 141, 142, 143, 144, 145, 146, 147, 152, 153, 154, 155, 158, 159, 160, 161, 164, 165, 168, 169, 170, 171, 174, 175, 176, 177, 182, 183, 184, 185, 186, 187, 188, 189, 194, 195]

[1, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]


In [2]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
empty=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if is_prime(index)==True:
        ppi=prime_pi(index)
        poly=s[k][1]
        cffs=poly.list()
        dz=dropzeros(cffs)
        dens=listdens(dz)
        pf= sort(primeFactorsInList(dens))
        if len(pf)==0:empty=empty+[index]
        if len(pf)>0:
            print (index,min(pf),max(pf))
print "empty: "
print empty
        
       

(2, 2, 3)
(3, 2, 2)
(5, 2, 3)
(7, 2, 5)
(11, 2, 7)
(13, 2, 11)
(17, 2, 13)
(19, 2, 17)
(23, 2, 19)
(29, 2, 23)
(31, 2, 29)
(37, 2, 31)
(41, 2, 37)
(43, 2, 41)
(47, 2, 43)
(53, 2, 47)
(59, 2, 53)
(61, 2, 59)
(67, 2, 61)
(71, 2, 67)
(73, 2, 71)
(79, 2, 73)
(83, 2, 79)
(89, 2, 83)
(97, 2, 89)
(101, 2, 97)
(103, 2, 101)
(107, 2, 103)
(109, 2, 107)
(113, 2, 109)
(127, 2, 113)
(131, 2, 127)
(137, 2, 131)
(139, 2, 137)
(149, 2, 139)
(151, 2, 149)
(157, 2, 151)
(163, 2, 157)
(167, 2, 163)
(173, 2, 167)
(179, 2, 173)
(181, 2, 179)
(191, 2, 181)
(193, 2, 191)
(197, 2, 193)
(199, 2, 197)
empty: 
[]


In [4]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
empty=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if is_prime(index)==False:
        if is_prime(index+1)==False:
            ppi=prime_pi(index)
            poly=s[k][1]
            cffs=poly.list()
            dz=dropzeros(cffs)
            dens=listdens(dz)
            pf= sort(primeFactorsInList(dens))
            if len(pf)==0:empty=empty+[index]
            if len(pf)>0:
                print (index,min(pf),max(pf))
print "empty: "
print empty

(0, 2, 2)
(8, 2, 7)
(9, 2, 7)
(14, 2, 13)
(15, 2, 13)
(20, 2, 19)
(21, 2, 19)
(24, 2, 23)
(25, 2, 23)
(26, 2, 23)
(27, 2, 23)
(32, 2, 31)
(33, 2, 31)
(34, 2, 31)
(35, 2, 31)
(38, 2, 37)
(39, 2, 37)
(44, 2, 43)
(45, 2, 43)
(48, 2, 47)
(49, 2, 47)
(50, 2, 47)
(51, 2, 47)
(54, 2, 53)
(55, 2, 53)
(56, 2, 53)
(57, 2, 53)
(62, 2, 61)
(63, 2, 61)
(64, 2, 61)
(65, 2, 61)
(68, 2, 67)
(69, 2, 67)
(74, 2, 73)
(75, 2, 73)
(76, 2, 73)
(77, 2, 73)
(80, 2, 79)
(81, 2, 79)
(84, 2, 83)
(85, 2, 83)
(86, 2, 83)
(87, 2, 83)
(90, 2, 89)
(91, 2, 89)
(92, 2, 89)
(93, 2, 89)
(94, 2, 89)
(95, 2, 89)
(98, 2, 97)
(99, 2, 97)
(104, 2, 103)
(105, 2, 103)
(110, 2, 109)
(111, 2, 109)
(114, 2, 113)
(115, 2, 113)
(116, 2, 113)
(117, 2, 113)
(118, 2, 113)
(119, 2, 113)
(120, 2, 113)
(121, 2, 113)
(122, 2, 113)
(123, 2, 113)
(124, 2, 113)
(125, 2, 113)
(128, 2, 127)
(129, 2, 127)
(132, 2, 131)
(133, 2, 131)
(134, 2, 131)
(135, 2, 131)
(140, 2, 139)
(141, 2, 139)
(142, 2, 139)
(143, 2, 139)
(144, 2, 139)
(145, 2, 139)
(1

In [5]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
empty=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    if is_prime(index)==False:
        if is_prime(index+1)==True:
            ppi=prime_pi(index)
            poly=s[k][1]
            cffs=poly.list()
            dz=dropzeros(cffs)
            dens=listdens(dz)
            pf= sort(primeFactorsInList(dens))
            if len(pf)==0:empty=empty+[index]
            if len(pf)>0:
                print (index,min(pf),max(pf))
print "empty: "

(1, 2, 2)
(4, 2, 5)
(6, 2, 7)
(10, 2, 11)
(12, 2, 13)
(16, 2, 17)
(18, 2, 19)
(22, 2, 23)
(28, 2, 29)
(30, 2, 31)
(36, 2, 37)
(40, 2, 41)
(42, 2, 43)
(46, 2, 47)
(52, 2, 53)
(58, 2, 59)
(60, 2, 61)
(66, 2, 67)
(70, 2, 71)
(72, 2, 73)
(78, 2, 79)
(82, 2, 83)
(88, 2, 89)
(96, 2, 97)
(100, 2, 101)
(102, 2, 103)
(106, 2, 107)
(108, 2, 109)
(112, 2, 113)
(126, 2, 127)
(130, 2, 131)
(136, 2, 137)
(138, 2, 139)
(148, 2, 149)
(150, 2, 151)
(156, 2, 157)
(162, 2, 163)
(166, 2, 167)
(172, 2, 173)
(178, 2, 179)
(180, 2, 181)
(190, 2, 191)
(192, 2, 193)
(196, 2, 197)
(198, 2, 199)
empty: 
